In [63]:
# import libaries

import csv
import json
import os.path

In [ ]:
# paths

raw_data_path = 'raw_data/'
search_path = 'json_files/search/search.json'

In [64]:
# Function: write_json
# Write a JSON file
# Purpose: Write a file from a dictonary in JSON format
# Input: Path to write out file, A list of dictonaries
# Output: A JSON formated file
# Restrictions: Writes the file with forced UTF-8 encoding

def write_json(file_path_out, list_dict):
    with open(file_path_out, 'w', encoding="utf8") as file_out:
        json.dump(list_dict, file_out)
    print(f'JSON file written to {file_path_out}')

In [65]:
# Function: read_csv
# Read a CSV file
# Purpose: Intake data that is in .csv format
# Input: Path to a singular file
# Output: Return a list of dictionaries of read CSV
# Restrictions: reads the file with forces UTF-8 encoding

def read_csv(file_path_in):
    with open(file_path_in, 'r', encoding="utf8") as file_in:
        file = csv.DictReader(file_in, delimiter=',')
        data_s0 = []
        for row in file:
            data_s0.append(row)
        data = data_s0
    data_name = os.path.splitext(os.path.basename(file_path_in))[0]
    file_path_out = f'json_files/data/{data_name}.json'
    write_json(file_path_out, data)

In [66]:
def convert_csv_json():
    file_list = os.listdir('raw_data/')
    for file in file_list:
        read_csv(f'raw_data/{file}')

In [67]:
# Function: read_json
# Read a JSON file
# Purpose: Intake data that is in .json format
# Input: Path to a singular file
# Output: Return a list of dictionaries of read JSON
# Restrictions: reads the file with forces UTF-8 encoding

def read_json(file_path):
    with open(file_path, 'r', encoding="utf8") as file_in:
        data_s0 = json.load(file_in)
        data = data_s0
    return data

In [91]:
# Function: search_filter
# Filter a JSON file
# Purpose: Filter a JSON file based on a search, and return a filtered JSON
# Input: A search JSON
# Output: A JSON formatted results file and console display 
# Restrictions: (blank)

def search_fliter(search_in):
    choice_data = search_in['data']
    choice_field = search_in['field']
    choice_value = search_in['value']
    choice_criteria = search_in['criteria']
    file_path_in = f'json_files/data/{choice_data}.json'
    json_in = read_json(file_path_in)
    if choice_criteria == 1:
        filtered_s0 = filter(lambda f: f[choice_field] == choice_value, json_in)
    elif choice_criteria == 2:
        filtered_s0 = filter(lambda f: f[choice_field] <= choice_value, json_in)
    elif choice_criteria == 3:
        filtered_s0 = filter(lambda f: f[choice_field] >= choice_value, json_in)
    else:
        filtered_s0 = json_in
    filtered = list(filtered_s0)
    file_path_out = f'json_files/output/{choice_data}_results.json'
    write_json(file_path_out, filtered)


In [69]:
def choose_data():
    file_list = os.listdir('json_files/data')
    files = [os.path.splitext(filename)[0] for filename in file_list]
    print('Which data do you want to search (get records from):')
    string = ""
    for i, name in enumerate(files):
        string_s0 = f'\t{i+1}. {name}\n'
        string += string_s0
    choice_data_s0 = int(input(string))
    choice_data = files[choice_data_s0-1]
    print(f"""Chose '{choice_data}' data""")
    return choice_data
    

In [70]:
def choose_field(choice_data):
    file_path_in = f'json_files/data/{choice_data}.json'
    fields = list(read_json(file_path_in)[0].keys())
    print('Which field do you want to search in "{choice_data}" data: ')
    string = ""
    for i, name in enumerate(fields):
        string_s0 = f'\t{i+1}. {name}\n'
        string += string_s0
    choice_field_s0 = int(input(string))
    choice_field = fields[choice_field_s0-1]
    print(f"""Chose field: {choice_field} from '{choice_data}' data""")
    return choice_field

In [71]:
def choose_critera(choice_data, choice_field, choice_value):
    choice_criteria = int(input(f""" 
    For value: "{choice_value}", What is the criteria for searching for in field: "{choice_field}" in "{choice_data}" data:
        1. equal to
        2. less than or equal to
        3. greater than or equal to
        4. all

        """))
    if 1 <= choice_criteria <= 3:
        if choice_criteria == 1:
            print(f"""Creating a search for data that is equal to "{choice_value}" in field: "{choice_field}" from "{choice_data}" data""")
        elif choice_criteria == 2:
            print(f"""Creating a search for data that is less than or equal to "{choice_value}" in field: "{choice_field}" from "{choice_data}" data""")
        elif choice_criteria == 3:
            print(f"""Creating a search for data that is greater than or equal to "{choice_value}" in field: "{choice_field}" from "{choice_data}" data""")

    else:
        choice_criteria = 'all'
        print(f"""Creating a search for all records from "{choice_data}" data""")
    return choice_criteria


In [78]:
# Function: create_search
# Create search criteria
# Purpose: Write a JSON formatted search file
# Input: User input
# Output: A JSON formmated file that can be used by the search() funcation 
# Restrictions: Writes the file with forced UTF-8 encoding, This funcation is not error correcting and may 
#               fail with certain user inputs

def create_search():
    choice_data = choose_data()
    
    choice_filter = int(input(f"""
    Do you want to filter "{choice_data}" 
        1. Yes
        2. No
    
    """))
    
    if choice_filter == 1:
        choice_field = choose_field(choice_data)
        choice_value = input(f""" 
    What is the value you are searching for in field: "{choice_field}" in "{choice_data}" data:

        """)
        choice_criteria = choose_critera(choice_data, choice_field, choice_value)
    else:
        choice_field = 'all'
        choice_value = 'all'
        choice_criteria = 'all'
        print(f"""Getting all records from "{choice_data}" data""")
            
    search = {'data': choice_data, 'field': choice_field, 'value': choice_value, 'criteria': choice_criteria}
    write_json(search_path, search)
    print(search)

In [73]:
# Function: search
# Search
# Purpose: filter JSON based on search.json
# Input: Expects criteria to be passed, if not looks for search.json in default location, 
#        if not found asks if search should be created 
# Output: A JSON formmated file that can be used by the search() funcation 
# Restrictions: (blank)

def search(choice_data = 'NE', choice_field = 'NE', choice_value= 'NE', choice_criteria = 'NE'):
    try:
        json_in = read_json(search_path)
        search_fliter(json_in)
    except:
        print("Cannot find search.json")
        choice_s = int(input("""
    Would you like to create a search?:
        1. Yes
        2. No
        
        """))
        if choice_s == 1:
            create_search()
            search()
        else:
            print("Goodbye")

In [84]:
def clear_search():
    if os.path.exists(search_path):
        os.remove(search_path)
        print('Search Cleared')
    else:
        print("Search already cleared")

In [74]:
convert_csv_json()

JSON file written to json_files/data/colors.json
JSON file written to json_files/data/elements.json
JSON file written to json_files/data/inventories.json
JSON file written to json_files/data/inventory_minifigs.json
JSON file written to json_files/data/inventory_parts.json
JSON file written to json_files/data/inventory_sets.json
JSON file written to json_files/data/minifigs.json
JSON file written to json_files/data/parts.json
JSON file written to json_files/data/part_categories.json
JSON file written to json_files/data/part_relationships.json
JSON file written to json_files/data/sets.json
JSON file written to json_files/data/themes.json


In [89]:
clear_search()

Search Cleared


In [92]:
search()

JSON file written to json_files/output/sets_results.json
